<a href="https://colab.research.google.com/github/yongsa-nut/SF323_CN408_AIEngineer/blob/main/Web_Search_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web Search Agent

---

Plans:
- Web Search APIs
- Web Search tools
- Simple Agentic Loop
- Clarification step
- Extra tools
- Multi agents

## Web Search APIs

We will be using tavily api.

Register and get the API from their [website](https://www.tavily.com/).

Read more here: https://docs.tavily.com/documentation/api-reference/endpoint/search

In [ ]:
# Install langchain for coding tool (later)
!pip install langchain_experimental

In [ ]:
!pip install tavily-python

In [ ]:
from tavily import TavilyClient
from google.colab import userdata
TAVILY_API = userdata.get('tavily') #change it to your tavily
tavily_client = TavilyClient(TAVILY_API)

#Search
response = tavily_client.search(
    query="ร้านส้มตำแถวธรมมศาสตร์รังสิค"
)
response

In [ ]:
response

In [ ]:
#Search
response = tavily_client.search(
    query="ร้านส้มตำแถวธรมมศาสตร์รังสิค",
    include_raw_content=True,
)
response

We can also use jina.ai to retrieve information.

Get your API [here](https://jina.ai/api-dashboard).

In [ ]:
import requests

JINA_API = userdata.get('jina') #change it to your tavily

url = 'https://r.jina.ai/https://www.wongnai.com/reviews/25a435fcc87143d5b48a1443f7a6cf01'
headers = {"Authorization": "Bearer "+JINA_API}

response = requests.get(url, headers=headers)

print(response.text)

## Web Search tools

First create a function to search and fetch only content


In [ ]:
def tavily_web_search(query: str,
                      max_results: int=5,
                      include_raw_content: bool=True) -> dict:
    response = tavily_client.search(
      query=query,
      max_results=max_results,
      include_raw_content=include_raw_content
    )
    return response

In [ ]:
result = tavily_web_search("ธรรมศาสตร์รังสิต")
result

In [ ]:
result['results'][2]

Search results can be very long and full of unrelated information.

We may want to extract/summarize only relevant information.

We will use a small LLM to do this (Note summary could result in losing information).

Prompt adapted from https://github.com/langchain-ai/deep_research_from_scratch/blob/main/src/deep_research_from_scratch/prompts.py#L136

In [71]:
from openai import OpenAI
from datetime import date
from typing_extensions import Annotated, List, Literal


openrouter_client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=userdata.get('openrouter'),
)

# A summarize function using an LLM. This is a simplified version which the content at 20000 words.
def summarize_webpage_content(query: str,
                              webpage_content: str,
                              model: str='google/gemini-2.5-flash',
                              max_words: int=20000) -> dict:
    today = date.today()
    summary_prompt = f"""Your task is to extract and summarize the raw content of a webpage retrieved from a web search related to a given query.
    This summary will be used by a downstream research agent, so it's crucial to maintain the key details without losing essential information.

    Here is the query:
    <query>
    {query}
    </query>

    Here is the raw content of the webpage:
    <webpage_content>
    {webpage_content[:max_words]}
    </webpage_content>

    Present your summary in the following format:

    ```
    {{
      "summary": "Your summary here, structured with appropriate paragraphs or bullet points as needed. Summary should include information related to query",
      "key_excerpts": "First important quote or excerpt, Second important quote or excerpt, Third important quote or excerpt, ...Add more excerpts as needed, up to a maximum of 5"
    }}
    ```

    Here are two examples of good summaries:

    Example 1 (for a news article):
    ```json
    {{
      "summary": "On July 15, 2023, NASA successfully launched the Artemis II mission from Kennedy Space Center. This marks the first crewed mission to the Moon since Apollo 17 in 1972. The four-person crew, led by Commander Jane Smith, will orbit the Moon for 10 days before returning to Earth. This mission is a crucial step in NASA's plans to establish a permanent human presence on the Moon by 2030.",
      "key_excerpts": "Artemis II represents a new era in space exploration, said NASA Administrator John Doe. The mission will test critical systems for future long-duration stays on the Moon, explained Lead Engineer Sarah Johnson. We're not just going back to the Moon, we're going forward to the Moon, Commander Jane Smith stated during the pre-launch press conference."
    }}
    ```

    Example 2 (for a scientific article):
    ```json
    {{
      "summary": "A new study published in Nature Climate Change reveals that global sea levels are rising faster than previously thought. Researchers analyzed satellite data from 1993 to 2022 and found that the rate of sea-level rise has accelerated by 0.08 mm/year² over the past three decades. This acceleration is primarily attributed to melting ice sheets in Greenland and Antarctica. The study projects that if current trends continue, global sea levels could rise by up to 2 meters by 2100, posing significant risks to coastal communities worldwide.",
      "key_excerpts": "Our findings indicate a clear acceleration in sea-level rise, which has significant implications for coastal planning and adaptation strategies, lead author Dr. Emily Brown stated. The rate of ice sheet melt in Greenland and Antarctica has tripled since the 1990s, the study reports. Without immediate and substantial reductions in greenhouse gas emissions, we are looking at potentially catastrophic sea-level rise by the end of this century, warned co-author Professor Michael Green."
    }}
    ```

    Remember, your goal is to create a summary that can be easily understood and utilized by a downstream research agent while preserving the most critical information from the original webpage.

    Today's date is {today}.
    """
    response = openrouter_client.chat.completions.create(
        model=model,
        messages=[{'role':'user','content':summary_prompt}]
    )
    return response.choices[0].message.content

def tavily_web_search_summary(query: str,
                      max_results: int=5,
                      include_raw_content: bool=True,
                      echo: bool=True) -> dict:
    summarized_results = {}
    results = tavily_web_search(query, max_results, include_raw_content)
    results = results['results']
    for result in results:
        if not result.get("raw_content"):
            content = result['content']
        else:
            # Summarize raw content for better processing
            content = summarize_webpage_content(query, result['raw_content'])

        summarized_results[result['url']] = {
           'title': result['title'],
           'content': content
        }
    if echo:
        print('---'*20)
        print(json.dumps(summarized_results, indent=4))
        print('---'*20)
    return summarized_results

def tavily_multiple_web_search(queries: List[str],
                               max_results: int=5,
                               include_raw_content: bool=True,
                               echo: bool=True) -> dict:
    results = {}
    for query in queries:
        result = tavily_web_search_summary(query, max_results, include_raw_content, echo)
        results[query] = result
    return results


In [ ]:
results = tavily_web_search_summary("Who is Leo Messi?")
results

Web Search agent

In [68]:
import json

# Tool defintion
web_search_tool = {
            "type": "function",
            "function":{
                "name": "web_search_tool",
                "description": "Fetch results from Tavily web search API with content summarization.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "A single search query to execute"
                        },
                        "max_results": {
                            "type": "int",
                            "description": "Maximum number of results to return"
                        }
                    },
                    "required": ["query"]
                }
            }
        }

web_search_multiple_tool = {
            "type": "function",
            "function":{
                "name": "web_search_multiple_tool",
                "description": "Fetch results from a list of queires using Tavily web search API with content summarization.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "queries": {
                            "type": "list",
                            "description": "A list of search queries"
                        },
                        "max_results": {
                            "type": "int",
                            "description": "Maximum number of results to return"
                        }
                    },
                    "required": ["queries"]
                }
            }
        }

def run_tool(name, arguments):
    if name == 'web_search_tool':
        return tavily_web_search_summary(**arguments)
    if name == 'web_search_multiple_tool':
        return tavily_multiple_web_search(**arguments)

def web_search_agent(query: str,
                     model: str='z-ai/glm-4.6') -> str:
    system_prompt = f"""You are a helpful research assistant agent. Your task is to answer user's query.
    You have accessed to a web search tool.
    Current date: {date.today()}
    """
    messages = [{'role':'system','content':system_prompt},
                  {'role':'user','content':query}]
    response = openrouter_client.chat.completions.create(
        model=model,
        messages=messages,
        tools = [web_search_tool]
    )
    if response.choices[0].finish_reason=='tool_calls':
        # call the tools
        messages.append(response.choices[0].message)
        tool_calls = response.choices[0].message.tool_calls
        tool_results = []
        for tool_call in tool_calls:
            tool_name = tool_call.function.name
            arguments = json.loads(tool_call.function.arguments)
            print(f'\033[92mTool\033[0m: {tool_name}({arguments})')
            tool_result = run_tool(tool_name, arguments)
            tool_result = {
                    "role" : "tool",
                    "tool_call_id": tool_call.id,
                    "name":tool_name,
                    "content" : str(tool_result)
            }
            tool_results.append(tool_result)
        messages.extend(tool_results)

        response = openrouter_client.chat.completions.create(
            model=model,
            messages=messages,
            tools = [web_search_tool]
        )

    return response.choices[0].message.content

In [ ]:
answer = web_search_agent('Who is CR7?')
print(answer)

In [77]:
def web_search_agent2(query: str,
                     model: str='alibaba/tongyi-deepresearch-30b-a3b') -> str:
    system_prompt = f"""You are a helpful research assistant agent. Your task is to answer user's query.
    You have accessed to a web search tool.
    Current date: {date.today()}
    """
    messages = [{'role':'system','content':system_prompt},
                  {'role':'user','content':query}]
    response = openrouter_client.chat.completions.create(
        model=model,
        messages=messages,
        tools = [web_search_multiple_tool]
    )

    if response.choices[0].finish_reason=='tool_calls':
        # call the tools
        messages.append(response.choices[0].message)
        tool_calls = response.choices[0].message.tool_calls
        tool_results = []
        for tool_call in tool_calls:
            tool_name = tool_call.function.name
            arguments = json.loads(tool_call.function.arguments)
            print(f'\033[92mTool\033[0m: {tool_name}({arguments})')
            tool_result = run_tool(tool_name, arguments)
            tool_result = {
                    "role" : "tool",
                    "tool_call_id": tool_call.id,
                    "name":tool_name,
                    "content" : str(tool_result)
            }
            tool_results.append(tool_result)
        messages.extend(tool_results)

        response = openrouter_client.chat.completions.create(
            model=model,
            messages=messages,
            tools = [web_search_tool]
        )

    return response.choices[0].message.content

In [76]:
answer = web_search_agent2('Who is CR7?')
print(answer)

Tool: web_search_multiple_tool({'queries': ['CR7 who is', 'Cristiano Ronaldo CR7 nickname origin', 'CR7 footballer biography']})
------------------------------------------------------------
{
    "https://www.ebsco.com/research-starters/biography/cristiano-ronaldo": {
        "title": "Cristiano Ronaldo | Research Starters",
        "content": "```json\n{\n  \"summary\": \"Cristiano Ronaldo dos Santos Aveiro, born on February 5, 1985, in Madeira, Portugal, is a renowned professional football (soccer) player known for his exceptional skill and record-breaking achievements. He began his career in local teams, quickly gaining recognition at Sporting Clube de Portugal before joining major clubs like Manchester United, Real Madrid, and Juventus. Ronaldo has accumulated numerous accolades, including multiple Ballon d'Or awards, and is the first player to win every major domestic trophy in England, Spain, and Italy.\\n\\nHe made history by scoring 800 career goals in 2021 and has continued to

---

<br>

## Agentic Web Search
1. Add a loop
2. Use a think tool to plan and reason between tool calls
3. Add coding tool

In [58]:
from langchain_experimental.utilities import PythonREPL
import warnings

warnings.filterwarnings("ignore")

In [38]:
# @title Tool Schema

web_search_tool = {
              "type": "function",
              "function":{
                  "name": "web_search_tool",
                  "description": "Fetch results from a list of queires using Tavily web search API with content summarization.",
                  "parameters": {
                      "type": "object",
                      "properties": {
                          "queries": {
                              "type": "list",
                              "description": "A list of search queries"
                          },
                          "max_results": {
                              "type": "int",
                              "description": "Maximum number of results to return. (Max is 5)"
                          }
                      },
                      "required": ["queries"]
                  }
              }
          }

think_tool = {
            "type": "function",
            "function":{
                "name": "think_tool",
                "description": """Tool for strategic reflection on research progress and decision-making.
    Use this tool after each search to analyze results and plan next steps systematically.

    Reflection should include:
    1. Analysis of current findings - What concrete information have I gathered?
    2. Gap assessment - What crucial information is still missing?
    3. Quality evaluation - Do I have sufficient evidence/examples for a good answer?
    4. Strategic decision - Should I continue searching or provide my answer?

    The function will returns a confirmation that reflection was recorded for decision-making.
                """,
                "parameters": {
                    "type": "object",
                    "properties": {
                        "reflection": {
                            "type": "str",
                            "description": "Your detailed reflection on research progress, findings, gaps, and next steps"
                        }
                    },
                    "required": ["reflection"]
                }
            }
        }
code_tool = {
            "type": "function",
            "function":{
                "name": "run_python_code",
                "description": "Execute python code. The code runs in a static sandbox without interactive mode, so print output.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "code": {
                            "type": "string",
                            "description": "Python code to execute."
                        }
                    },
                    "required": ["code"]
                }
            }
        }

In [56]:
# @title prompts
from datetime import date

search_agent_prompt =  f"""You are a search agent conducting research on the user's input topic. For context, today's date is {date.today()}.
You have access to web_search_tool. Use it to gather information about the user's input topic.
You may use the web_search_tool multiple times until you get the results that you want.

<Instructions>
Think like a human researcher with limited time. Follow these steps:

1. **Read the question carefully** - What specific information does the user need?
2. **Start with broader searches** - Use broad, comprehensive queries first
3. **After each search, pause and assess** - Do I have enough to answer? What's still missing?
4. **Execute narrower searches as you gather information** - Fill in the gaps
5. **Stop when you can answer confidently** - Don't keep searching for perfection
</Instructions>

<Hard Limits>
**Tool Call Budgets** (Prevent excessive searching):
- **Simple queries**: Use 2-3 search tool calls maximum
- **Complex queries**: Use up to 5 search tool calls maximum
- **Always stop**: After 5 search tool calls if you cannot find the right sources

**Stop Immediately When**:
- You can answer the user's question comprehensively
- You have 3+ relevant examples/sources for the question
- Your last 2 searches returned similar information
</Hard Limits>
"""

research_agent_prompt =  f"""You are a research assistant conducting research on the user's input topic. For context, today's date is {date.today()}.

<Task>
Your job is to use tools to gather information about the user's input topic.
You can use any of the tools provided to you to find resources that can help answer the research question.
</Task>

<Available Tools>
You have access to two main tools:
1. **web_search_tool**: For conducting web searches to gather information.
2. **think_tool**: For reflection and strategic planning during research

**CRITICAL: Use think_tool after each search to reflect on results and plan next steps**
</Available Tools>

<Instructions>
Think like a human researcher with limited time. Follow these steps:

1. **Read the question carefully** - What specific information does the user need?
2. **Start with broader searches** - Use broad, comprehensive queries first
3. **After each search, pause and assess** - Do I have enough to answer? What's still missing?
4. **Execute narrower searches as you gather information** - Fill in the gaps
5. **Stop when you can answer confidently** - Don't keep searching for perfection
</Instructions>

<Hard Limits>
**Tool Call Budgets** (Prevent excessive searching):
- **Simple queries**: Use 2-3 search tool calls maximum
- **Complex queries**: Use up to 5 search tool calls maximum
- **Always stop**: After 5 search tool calls if you cannot find the right sources

**Stop Immediately When**:
- You can answer the user's question comprehensively
- You have 3+ relevant examples/sources for the question
- Your last 2 searches returned similar information
</Hard Limits>

<Show Your Thinking>
After each search tool call, use think_tool to analyze the results:
- What key information did I find?
- What's missing?
- Do I have enough to answer the question comprehensively?
- Should I search more or provide my answer?
</Show Your Thinking>
"""

code_research_agent_prompt =  f"""You are a research assistant conducting research on the user's input topic. For context, today's date is {date.today()}.

<Task>
Your job is to use tools to gather information about the user's input topic.
You can use any of the tools provided to you to find resources that can help answer the research question.
</Task>

<Available Tools>
You have access to three main tools:
1. **web_search_tool**: For conducting web searches to gather information.
2. **think_tool**: For reflection and strategic planning during research
3. **run_python_code**: For running python script to calculate things.
**CRITICAL: Use think_tool after each search to reflect on results and plan next steps**
</Available Tools>

<Instructions>
Think like a human researcher with limited time. Follow these steps:

1. **Read the question carefully** - What specific information does the user need?
2. **Start with broader searches** - Use broad, comprehensive queries first
3. **After each search, pause and assess** - Do I have enough to answer? What's still missing?
4. **Execute narrower searches as you gather information** - Fill in the gaps
5. **Stop when you can answer confidently** - Don't keep searching for perfection
</Instructions>

<Hard Limits>
**Tool Call Budgets** (Prevent excessive searching):
- **Simple queries**: Use 2-3 search tool calls maximum
- **Complex queries**: Use up to 5 search tool calls maximum
- **Always stop**: After 5 search tool calls if you cannot find the right sources

**Stop Immediately When**:
- You can answer the user's question comprehensively
- You have 3+ relevant examples/sources for the question
- Your last 2 searches returned similar information
</Hard Limits>

<Show Your Thinking>
After each search tool call, use think_tool to analyze the results:
- What key information did I find?
- What's missing?
- Do I have enough to answer the question comprehensively?
- Should I search more or provide my answer?
</Show Your Thinking>
"""

In [59]:
import json

class WebSearchAgent:

    def __init__(self, system_prompt, tools, model = "z-ai/glm-4.6"):
        # LLM setup
        self.client = OpenAI(
            base_url="https://openrouter.ai/api/v1",
            api_key=userdata.get('openrouter'),
        )
        self.model = model
        self.tavily_client = TavilyClient(userdata.get('tavily'))
        self.python_repl = PythonREPL()
        self.messages = [{'role':'system','content':system_prompt}]
        self.tools = tools

    ## --- Web Search functions --- ##

    def tavily_web_search(self, query: str,
                      max_results: int=5,
                      include_raw_content: bool=True) -> dict:
        response = self.tavily_client.search(
          query=query,
          max_results=max_results,
          include_raw_content=include_raw_content
        )
        return response

    def summarize_webpage_content(self, query: str,
                              webpage_content: str,
                              model: str='google/gemini-2.5-flash',
                              max_words: int=20000) -> dict:
        summary_prompt = f"""Your task is to extract and summarize the raw content of a webpage retrieved from a web search related to a given query.
        This summary will be used by a downstream research agent, so it's crucial to maintain the key details without losing essential information.

        Here is the query:
        <query>
        {query}
        </query>

        Here is the raw content of the webpage:
        <webpage_content>
        {webpage_content[:max_words]}
        </webpage_content>

        Present your summary in the following format:

        ```
        {{
          "summary": "Your summary here, structured with appropriate paragraphs or bullet points as needed. Summary should include information related to query",
          "key_excerpts": "First important quote or excerpt, Second important quote or excerpt, Third important quote or excerpt, ...Add more excerpts as needed, up to a maximum of 5"
        }}
        ```

        Here are two examples of good summaries:

        Example 1 (for a news article):
        ```json
        {{
          "summary": "On July 15, 2023, NASA successfully launched the Artemis II mission from Kennedy Space Center. This marks the first crewed mission to the Moon since Apollo 17 in 1972. The four-person crew, led by Commander Jane Smith, will orbit the Moon for 10 days before returning to Earth. This mission is a crucial step in NASA's plans to establish a permanent human presence on the Moon by 2030.",
          "key_excerpts": "Artemis II represents a new era in space exploration, said NASA Administrator John Doe. The mission will test critical systems for future long-duration stays on the Moon, explained Lead Engineer Sarah Johnson. We're not just going back to the Moon, we're going forward to the Moon, Commander Jane Smith stated during the pre-launch press conference."
        }}
        ```

        Example 2 (for a scientific article):
        ```json
        {{
          "summary": "A new study published in Nature Climate Change reveals that global sea levels are rising faster than previously thought. Researchers analyzed satellite data from 1993 to 2022 and found that the rate of sea-level rise has accelerated by 0.08 mm/year² over the past three decades. This acceleration is primarily attributed to melting ice sheets in Greenland and Antarctica. The study projects that if current trends continue, global sea levels could rise by up to 2 meters by 2100, posing significant risks to coastal communities worldwide.",
          "key_excerpts": "Our findings indicate a clear acceleration in sea-level rise, which has significant implications for coastal planning and adaptation strategies, lead author Dr. Emily Brown stated. The rate of ice sheet melt in Greenland and Antarctica has tripled since the 1990s, the study reports. Without immediate and substantial reductions in greenhouse gas emissions, we are looking at potentially catastrophic sea-level rise by the end of this century, warned co-author Professor Michael Green."
        }}
        ```

        Remember, your goal is to create a summary that can be easily understood and utilized by a downstream research agent while preserving the most critical information from the original webpage.

        Today's date is {date.today()}.
        """
        response = openrouter_client.chat.completions.create(
            model=model,
            messages=[{'role':'user','content':summary_prompt}]
        )
        return response.choices[0].message.content

    def tavily_web_search_summary(self, query: str,
                          max_results: int=5,
                          include_raw_content: bool=True,
                          echo: bool=True) -> dict:
        summarized_results = {}
        results = self.tavily_web_search(query, max_results, include_raw_content)
        results = results['results']
        for result in results:
            if not result.get("raw_content"):
                content = result['content']
            else:
                # Summarize raw content for better processing
                content = self.summarize_webpage_content(query, result['raw_content'])

            summarized_results[result['url']] = {
              'title': result['title'],
              'content': content
            }
        if echo:
            print('---'*20)
            print(json.dumps(summarized_results, indent=4))
            print('---'*20)
        return summarized_results

    def tavily_multiple_web_search_tool(self, queries: List[str],
                                  max_results: int=5,
                                  include_raw_content: bool=True,
                                  echo: bool=True) -> dict:
        results = {}
        for query in queries:
            result = self.tavily_web_search_summary(query, max_results, include_raw_content, echo)
            results[query] = result
        return results

    ## --- other tools --- ##

    def think_tool(self, reflection: str) -> str:
        return f"Reflection recorded: {reflection}"

    def run_python_code(self, code: str) -> str:
        try:
            # Execute code directly (environment already set up)
            result = self.python_repl.run(code)
            print(f"Code executed successfully:\nOutput:\n{result}")
            return f"Code executed successfully:\nOutput:\n{result}"
        except Exception as e:
            return f"Error executing code:\nError: {str(e)}"

    def run_tool(self, name, arguments):
        if name == 'web_search_tool':
            return self.tavily_multiple_web_search_tool(**arguments)
        if name == ' think_tool':
            return self.think_tool(**arguments)
        if name == 'run_python_code':
            return self.run_python_code(**arguments)

    def run_tools(self, tool_calls):
        tool_results = []
        for tool_call in tool_calls:
            try:
                tool_name = tool_call.function.name
                arguments = json.loads(tool_call.function.arguments)
                print(f'\033[92mTool\033[0m: {tool_name}({arguments})')
                tool_result = self.run_tool(tool_name, arguments)
                tool_result = {
                        "role" : "tool",
                        "tool_call_id": tool_call.id,
                        "name":tool_name,
                        "content" : str(tool_result)
                }
            except Exception as e:
                tool_result = {
                        "role" : "tool",
                        "tool_call_id": tool_call.id,
                        "name":tool_name,
                        "content" : f"Error: {e}",
                }
            tool_results.append(tool_result)

        return tool_results

    def run(self, query):
        self.messages.append({'role': 'user', 'content': query})

        # Tool Handling
        while True:
            response = self.client.chat.completions.create(
                model = self.model,
                messages = self.messages,
                tools = self.tools
            )
            print('reasoning: ',response.choices[0].message.reasoning)
            if response.choices[0].finish_reason != 'tool_calls':
                break
            if response.choices[0].message.content != '': print('\033[38;5;208mAssistant\033[0m:', response.choices[0].message.content)
            self.messages.append(response.choices[0].message)
            results = self.run_tools(response.choices[0].message.tool_calls)
            self.messages.extend(results)

        # Final response (per turn)
        self.messages.append({'role': 'assistant',
                              'content': response.choices[0].message.content})
        print('\033[38;5;208mAssistant\033[0m:', response.choices[0].message.content)

In [ ]:
# Basic Search Agent
base_search_agent = WebSearchAgent(search_agent_prompt, [web_search_tool])
base_search_agent.run("Do a reserach on CR7")

In [ ]:
base_search_agent.run("Do a reserach on CR7")

In [ ]:
# Research Agent with think tool
research_agent = WebSearchAgent(research_agent_prompt, [web_search_tool, think_tool])
research_agent.run("Do a reserach on CR7")

In [50]:
research_agent.run("Do research on GDP of South East Asian countries in 2024")

Assistant: 

reasoning:  None
Tool: web_search_tool({'queries': ['GDP Southeast Asian countries 2024 economic data statistics'], 'max_results': 5})
------------------------------------------------------------
{
    "https://ustr.gov/countries-regions/southeast-asia-pacific/association-southeast-asian-nations-asean": {
        "title": "Association of Southeast Asian Nations (ASEAN) - USTR",
        "content": "```json\n{\n  \"summary\": \"The Association of Southeast Asian Nations (ASEAN) comprises ten countries: Brunei Darussalam, Burma, Cambodia, Indonesia, Laos, Malaysia, Philippines, Singapore, Thailand, and Vietnam. These nations collectively represent a market with an approximate GDP of $3.9 trillion and a population of 678 million people. The United States maintains a strong trade and investment relationship with ASEAN.\\n\\nIn 2024, the total U.S. goods and services trade with ASEAN was estimated at $571.7 billion, showing a 13.4% increase from 2023. Specifically, U.S. total go

In [60]:
# Research Agent with think tool and code tool
code_research_agent = WebSearchAgent(code_research_agent_prompt, [web_search_tool, think_tool, code_tool])
code_research_agent.run("What are the GDP of Thailand between 2022-2024. What are the avg GDP across three years?")

reasoning:  The user is asking for GDP data for Thailand from 2022-2024, and wants the average across those three years. This is a straightforward factual question that requires economic data.

I need to:
1. Search for Thailand's GDP data for 2022, 2023, and 2024
2. Calculate the average across the three years

Let me start with a broad search that covers Thailand's GDP over this period.
Assistant: 

I'll help you find Thailand's GDP data for 2022-2024 and calculate the average. Let me start by searching for this economic information.

Tool: web_search_tool({'queries': ['Thailand GDP 2022 2023 2024 economic data statistics', 'Thailand gross domestic product annual figures 2022-2024'], 'max_results': 5})
------------------------------------------------------------
{
    "https://tradingeconomics.com/thailand/gdp": {
        "title": "Thailand GDP - Trading Economics",
        "content": "```json\n{\n  \"summary\": \"According to official data from the World Bank, Thailand's GDP was valu

## Add a clarification step


In [40]:
class ClarificationAgent:
    def __init__(self, model = "google/gemini-2.5-flash-preview-09-2025"):
        # LLM setup
        self.client = OpenAI(
            base_url="https://openrouter.ai/api/v1",
            api_key=userdata.get('openrouter'),
        )
        self.model = model
        system_prompt = "You are a helpful assistant. Your task is to ask clarification question if user's question is unclear or required further details."
        self.messages = [{'role':'system','content':system_prompt}]

    def run(self):

        user_query = input('User: ')
        clarification_prompt = f"""Below is user's initial query.
<Messages>
{user_query}
</Messages>

Today's date is {date.today()}.

Assess whether you need to ask a clarifying question, or if the user has already provided enough information for you to start research.
**IMPORTANT**: If you can see in the messages history that you have already asked a clarifying question, you almost always do not need to ask another one. Only ask another question if ABSOLUTELY NECESSARY.

If there are acronyms, abbreviations, or unknown terms, ask the user to clarify.
If you need to ask a question, follow these guidelines:
- Be concise while gathering all necessary information
- Make sure to gather all the information needed to carry out the research task in a concise, well-structured manner.
- Use bullet points or numbered lists if appropriate for clarity. Make sure that this uses markdown formatting and will be rendered correctly if the string output is passed to a markdown renderer.
- Don't ask for unnecessary information, or information that the user has already provided. If you can see that the user has already provided the information, do not ask for it again.

Respond in valid JSON format with these exact keys:
"need_clarification": boolean (True or False),
"question": "<question to ask the user to clarify the report scope>",
"final_question": "<final question with all details to be sent to a research agent. Format it nicely with bullet points as appropriated>"

If you need to ask a clarifying question, return:
"need_clarification": True,
"question": "<your clarifying question>",
"final_question": ""

If you do not need to ask a clarifying question, return:
"need_clarification": False,
"question": "",
"final_question": "<final question with all details>"
        """
        self.messages.append({'role': 'user', 'content': clarification_prompt})

        while True:
            response = self.client.chat.completions.create(
                        model = self.model,
                        messages = self.messages,
                    )
            print(response.choices[0].message.content)
            response = json.loads(response.choices[0].message.content.replace('```json','').replace('```',''))

            if response['need_clarification']:
                self.messages.append({'role': 'assistant', 'content': response['question']})
                print('\033[38;5;208mAssistant\033[0m:',response['question'])
                user_query = input('User: ')
                self.messages.append({'role': 'user', 'content': user_query})
            else:
                print(response['final_question'])
                return response['final_question']


In [ ]:
# Testing the clarification
a = ClarificationAgent()
a.run()

In [ ]:
# Putting it all together
final_query = a.run()
code_research_agent.run(final_query)

## Multi-Agent System

Plan:
- Main agent
- Sub agents

## Sub Agents:

- Sub agent that will conduct research for you.
- Sub agent will take in a research prompt and return a brief
- Sub agent is essentially a function call for the main agent.
- Sub agent is similar to what the web search agent above.


In [112]:
web_search_multiple_tool = {
            "type": "function",
            "function":{
                "name": "web_search_multiple_tool",
                "description": "Fetch results from a list of queires using Tavily web search API with content summarization.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "queries": {
                            "type": "list",
                            "description": "A list of search queries"
                        },
                        "max_results": {
                            "type": "int",
                            "description": "Maximum number of results to return (max = 5)"
                        }
                    },
                    "required": ["queries"]
                }
            }
        }

def web_search_subagent(query: str,
                     model: str='z-ai/glm-4.6') -> str:
    system_prompt = f"""You are a helpful research assistant agent. You will be given a research task and your goal is to return a brief answering the research question.
    Your brief should be comprehensive and stay on the topic.
    You have accessed to a web search tool.
    Current date: {date.today()}
    """
    print(f'\033[35mSub agent\033[0m: query = {query}')
    messages = [{'role':'system','content':system_prompt},
                  {'role':'user','content':query}]

    while True:
        response = openrouter_client.chat.completions.create(
            model = model,
            messages = messages,
            tools = [web_search_multiple_tool]
        )
        if response.choices[0].finish_reason != 'tool_calls':
            break
        messages.append(response.choices[0].message)
        tool_calls = response.choices[0].message.tool_calls
        tool_results = []
        for tool_call in tool_calls:
            tool_name = tool_call.function.name
            arguments = json.loads(tool_call.function.arguments)
            print(f'\033[95mTool\033[0m: {tool_name}({arguments})')
            tool_result = tavily_multiple_web_search(**arguments)
            tool_result = {
                    "role" : "tool",
                    "tool_call_id": tool_call.id,
                    "name":tool_name,
                    "content" : str(tool_result)
            }
            tool_results.append(tool_result)
        messages.extend(tool_results)

    return response.choices[0].message.content

In [77]:
# Testing our subagent
web_search_subagent("Do a search on Messi")

Tool: web_search_multiple_tool({'queries': ['Lionel Messi 2024 2025 current news', 'Messi Inter Miami performance 2024', 'Lionel Messi recent achievements 2024'], 'max_results': 5})
------------------------------------------------------------
{
    "https://dailysports.net/news/messi-i-will-return-to-barcelona-but-only-when-he-leaves/": {
        "title": "Messi: \"I will return to Barcelona, but only when he leaves\"",
        "content": "Lionel Messi's contract with Inter Miami runs until December 31, 2025, and he has yet to extend it, although many speculate that he will"
    },
    "https://www.espn.com/soccer/story/_/id/43935337/lionel-messi-2025-tracker-inter-miami-games-goals-assists-stats": {
        "title": "Messi tracker: Goals, assists, key moments for Inter Miami",
        "content": "```json\n{\n  \"summary\": \"This webpage provides a detailed tracker of Lionel Messi's performance for Inter Miami CF during his 2025 season. As of October 11, 2025, Messi has played 42 game

'\n\nCurrent club: Inter Miami CF (MLS), contract runs until Dec 31 2025  \n\n2024 season: 20 goals and 16 assists in 19 MLS matches, led Inter Miami to first Supporters’ Shield, captained Argentina to Copa América title, tied Landon Donovan’s men’s international assist record (58)  \n\n2025 season (as of Oct 11 2025): 34 goals, 20 assists in 42 games, leading MLS Golden Boot race, 44 goal contributions (second most in MLS single‑season history), joined Carlos Vela as only players with 40+ contributions in a season  \n\nHonors: 8 Ballon d’Or awards, 4 The Best FIFA Men’s Player, 2022 World Cup champion, 2 Copa América, 4 UEFA Champions League, multiple scoring records  \n\nTransfer outlook: Messi said he would return to Barcelona only after leaving Inter Miami; no contract extension yet beyond 2025  \n\nImpact: Elevated Inter Miami’s profile, contributed to MLS growth, maintained elite performance at age 37'

### Main Agent:

- Main agent has the following tools:
  - `conduct_research('prompt')`: Call the subagent to do a research for you.
  - `read_file(file_name)`: read file. use it to read the draft of the research
  - `edit_file(file_name, old_str, new_str)`: edit the file.
  - `think_tool()`: reflect the progress so far both in terms of search and the draft.
  - `research_complete`: Call when the research is complete.

- Main agent workflow:
  1. Create a draft file with an initial layout and section in it.
  2. Begin searching.
  3. Use `think_tool` to think about the search
  4. Update the draft.
  5. Use `think_tool` to check if the draft is done.
  6. If the draft is done, use `done()`. If not, go back to search again.

In [111]:
# @title Tool schema
conduct_research_tool = {
              "type": "function",
              "function":{
                  "name": "conduct_research",
                  "description": "Fetch results from a list of queires using Tavily web search API with content summarization.",
                  "parameters": {
                      "type": "object",
                      "properties": {
                          "query": {
                              "type": "str",
                              "description": "A search query or instruction for the subagent"
                          }
                      },
                      "required": ["queries"]
                  }
              }
          }

research_complete_tool = {
            "type": "function",
            "function":{
                "name": "research_complete",
                "description": "Use this tool when you finish the research",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "summary_message": {
                            "type": "str",
                            "description": "Summary of the research to be sent to the user."
                        }
                    },
                    "required": ["summary_message"]
                }
            }
        }

think_tool = {
            "type": "function",
            "function":{
                "name": "think_tool",
                "description": """Tool for strategic reflection on research progress and decision-making.
    Use this tool after each search to analyze results and plan next steps systematically.

    Reflection should include:
    1. Analysis of current findings - What concrete information have I gathered?
    2. Gap assessment - What crucial information is still missing?
    3. Quality evaluation - Do I have sufficient evidence/examples for a good answer?
    4. Strategic decision - Should I continue searching or provide my answer?

    The function will returns a confirmation that reflection was recorded for decision-making.
                """,
                "parameters": {
                    "type": "object",
                    "properties": {
                        "reflection": {
                            "type": "str",
                            "description": "Your detailed reflection on research progress, findings, gaps, and next steps"
                        }
                    },
                    "required": ["reflection"]
                }
            }
        }
edit_file = {
            "type": "function",
            "function": {
                "name": "edit_file",
                "description": "Make edits to a text file. Replaces 'old_str' with 'new_str' in the given file. 'old_str' and 'new_str' MUST be different from each other. If the file specified with path doesn't exist, it will be created.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "file_path": {
                            "type": "string",
                            "description": "The path to the file you want to edit."
                        },
                        "old_str": {
                            "type": "string",
                            "description": "The string to be replaced in the file."
                        },
                        "new_str": {
                            "type": "string",
                            "description": "The string to replace the old string with."
                        }
                    },
                    "required": ["file_path", "old_str", "new_str"]
                }
            }
        }
read_file = {
            "type": "function",
            "function":{
                "name": "read_file",
                "description": "Read a file and return its content as a string",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "filename": {
                            "type": "string",
                            "description": "The name of the file you want to read."
                        }
                    },
                    "required": ["filename"]
                }
            }
        }

In [110]:
# @title Main agent prompt

main_agent_prompt = f"""You are a research supervisor. Your job is to conduct research by calling the "conduct_research" tool.
For context, today's date is {date.today()}.

To complete this task, follow the following step strictly:
<Step>
1. You begin by creating a draft file with an initial layout and sections in it. The draft should answer user's research questions. This initial draft will be your guiding plan but not a definite plan.
2. Use the "conduct_research" tool to conduct research.
3. Use think_tool to think and synthesize about the findings.
4. Update the draft based on findings and your thinking. The layout of the draft may change based on the findings.
5. Use think_tool to check if the draft progress and plan for the next step.
6. If the draft is done, you should call 'research_complete'. If not, go back to use the "conduct_research" tool to continue researching.
</Step>

<Available Tools>
You have access to three main tools:
1. **conduct_research**: Delegate research tasks to specialized sub-agents. Think carefully how you want to phrase the query. Your query should include all necessary information.
2. **research_complete**: Indicate that research is complete
3. **think_tool**: For reflection and strategic planning during research
4. **read_file**: For reading the current draft
5. **edit_file**: For editing and creating a new draft. Only edit the draft file. The edit file requires exactly three arguments: 'file_path', 'old_str', 'new_str'.
</Available Tools>

<Instructions>
Think like a research manager with limited time and resources. Follow these steps:

1. **Read the question carefully** - What specific information does the user need?
2. **Decide how to delegate the research** - Carefully consider the question and decide how to delegate the research. Are there multiple independent directions that can be explored simultaneously?
3. **After each call to conduct_research, pause and assess** - Do I have enough to answer? What's still missing?
</Instructions>

<Hard Limits>
**Task Delegation Budgets** (Prevent excessive delegation):
- **Bias towards single agent** - Use single agent for simplicity unless the user request has clear opportunity for parallelization
- **Stop when you can answer confidently** - Don't keep delegating research for perfection
- **Limit tool calls** - Always stop after 5 tool calls to think_tool and conduct_research if you cannot find the right sources
</Hard Limits>

<Show Your Thinking>
After each conduct_research tool call, use think_tool to analyze the results:
- What key information did I find?
- What's missing?
- Do I have enough to answer the question comprehensively?
- Should I delegate more research or call research_complete?
</Show Your Thinking>

<Scaling Rules>
**Simple fact-finding, lists, and rankings** can use a single sub-agent:
- *Example*: List the top 10 coffee shops in San Francisco → Use 1 sub-agent

**Comparisons presented in the user request** can use a sub-agent for each element of the comparison:
- *Example*: Compare OpenAI vs. Anthropic vs. DeepMind approaches to AI safety → Use 3 sub-agents
- Delegate clear, distinct, non-overlapping subtopics

**Important Reminders:**
- Each conduct_research call spawns a dedicated research agent for that specific topic
- A separate agent will write the final report - you just need to gather information
- When calling conduct_research, provide complete standalone instructions - sub-agents can't see other agents' work
- Do NOT use acronyms or abbreviations in your research questions, be very clear and specific
</Scaling Rules>"""

In [113]:
# @title main agent

import json

class WebSearchMultiAgent:

    def __init__(self, model = "z-ai/glm-4.6"):
        # LLM setup
        self.client = OpenAI(
            base_url="https://openrouter.ai/api/v1",
            api_key=userdata.get('openrouter'),
        )
        self.model = model
        self.messages = [{'role':'system','content':main_agent_prompt}]
        self.tools = [conduct_research_tool,
                      research_complete_tool,
                      think_tool,
                      read_file,
                      edit_file]

    def read_file(self, filename):
        try:
            with open(filename, 'r') as file:
                return file.read()
        except Exception as e:
            return (f"Error reading file: {e}")

    def edit_file(self, file_path, old_str, new_str):
        try:
            # Try to read existing file, create empty content if file doesn't exist
            try:
                with open(file_path, 'r', encoding='utf-8') as file:
                    content = file.read()
            except FileNotFoundError:
                content = ""

            # Check if old_str exists in content
            if old_str not in content:
                return f"No occurrences of the specified text found in '{file_path}'. No changes made."

            # Replace old_str with new_str
            modified_content = content.replace(old_str, new_str, 1)

            # Write the modified content back to the file
            with open(file_path, 'w', encoding='utf-8') as file:
                file.write(modified_content)

            # Count replacements made
            replacements = content.count(old_str)
            if replacements > 0:
                return f"Successfully replaced {replacements} occurrence(s) of '{old_str}' with '{new_str}' in '{file_path}'."
            else:
                return f"No occurrences of '{old_str}' found in '{file_path}'. File created/updated."

        except Exception as e:
            return f"Error editing file '{file_path}': {e}"

    def think_tool(self, reflection: str) -> str:
        return f"Reflection recorded: {reflection}"

    def run_tool(self, name, arguments):
        if name == 'conduct_research':
            return web_search_subagent(**arguments) # This one is outside function
        if name == ' think_tool':
            return self.think_tool(**arguments)
        if name == 'read_file':
            return self.read_file(**arguments)
        if name == 'edit_file':
            return self.edit_file(**arguments)

    def run_tools(self, tool_calls):
        tool_results = []
        for tool_call in tool_calls:
            try:
                tool_name = tool_call.function.name
                arguments = json.loads(tool_call.function.arguments)
                print(f'\033[92mTool\033[0m: {tool_name}({arguments})')
                tool_result = self.run_tool(tool_name, arguments)
                tool_result = {
                        "role" : "tool",
                        "tool_call_id": tool_call.id,
                        "name":tool_name,
                        "content" : str(tool_result)
                }
            except Exception as e:
                tool_result = {
                        "role" : "tool",
                        "tool_call_id": tool_call.id,
                        "name":tool_name,
                        "content" : f"Error: {e}",
                }
            tool_results.append(tool_result)

        return tool_results

    def run(self, query):
        self.messages.append({'role': 'user', 'content': query})

        # Tool Handling
        while True:
            response = self.client.chat.completions.create(
                model = self.model,
                messages = self.messages,
                tools = self.tools
            )
            #print('reasoning: ',response.choices[0].message.reasoning)
            if response.choices[0].finish_reason != 'tool_calls' :
                print('\033[38;5;208mAssistant\033[0m:',
                      response.choices[0].message.content)
                return
            if response.choices[0].message.tool_calls[0].function.name == 'research_complete':
                print(f'\033[92mTool\033[0m: research_complete)')
                print('\033[38;5;208mAssistant\033[0m:',
                      json.loads(response.choices[0].message.tool_calls[0].function.arguments)['summary_message'])
                return

            self.messages.append(response.choices[0].message)
            results = self.run_tools(response.choices[0].message.tool_calls)
            self.messages.extend(results)

In [ ]:
# Testing the multi agent

multi_agent = WebSearchMultiAgent()
multi_agent.run("Do a research on CR7")

In [114]:
# Another example
multi_agent = WebSearchMultiAgent()
multi_agent.run("Conduct a research on Thailand's GDP in 2024.")

Tool: edit_file({'file_path': 'thailand_gdp_2024_draft.md', 'old_str': '', 'new_str': "# Thailand's GDP in 2024: Research Report\n\n## Overview\n- Executive Summary\n- Key GDP Figures for 2024\n\n## Economic Performance\n- GDP Growth Rate\n- GDP Value (in USD and Thai Baht)\n- Quarterly Performance\n- Sector-wise Contributions\n\n## Context and Analysis\n- Comparison with 2023 Performance\n- Regional Standing (ASEAN/Asia)\n- Key Economic Drivers\n- Challenges and Constraints\n\n## Economic Factors Impacting GDP\n- Tourism Sector Performance\n- Export Performance\n- Domestic Consumption\n- Investment Trends\n- Government Policies\n\n## Outlook and Projections\n- 2024 Economic Outlook\n- Growth Forecasts\n- Risk Factors\n\n## Sources and Methodology"})
Tool: conduct_research({'query': 'Thailand GDP 2024 economic performance growth rate official statistics data quarterly results'})
Sub agent: query = Thailand GDP 2024 economic performance growth rate official statistics data quarterly res